# <span style="background-color:white;color:red;"> ⌛⌛ EN PROGRESO. NO ES ÚTIL⌛⌛</span>

## Bloque I
Descomprimir el csv en formato gz

In [ ]:
"""  
Importación de ficheros de AIRBNB. Preprocesado y cargado a la base de datos 

Obtenemos los datos desde https://insideairbnb.com/get-the-data/

Podríamos hacer scraping, obtener la fecha de la última actualización de Madrid y generar el enlace para descargarlo, pero no es necesario por que se actualiza cada mucho tiempo y no es necesario tener la última versión. Podemos cambiar la info procesable en la carpeta data manualmente.

"""
import os
import pandas as pd
import numpy as np
import datetime
from config_bd import *
import requests
import sys
import gzip
import shutil

DATA_DIR = os.path.join('data')
# DATA_DIR = os.path.join(os.path.dirname(__file__), 'data')
TEMP_DIR = os.path.join(DATA_DIR, 'TEMP')


""" Comprobamos que existe el directorio de datos """

if not os.path.exists(DATA_DIR):
    print(f"El directorio {DATA_DIR} no existe. No es posible continuar...")
    sys.exit()

""" Descomprimimos el archivo de calendario """

if not os.path.exists(TEMP_DIR):
    os.makedirs(TEMP_DIR)


gz_file_path = os.path.join(DATA_DIR, 'calendar.csv.gz')
output_file_path = os.path.join(TEMP_DIR, os.path.splitext(os.path.basename(gz_file_path))[0])

if os.path.exists(gz_file_path):
    try:
        with gzip.open(gz_file_path, 'rb') as f_in:
            with open(output_file_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Archivo {gz_file_path} descomprimido como {output_file_path}")
    except Exception as e:
        print(f"Error al descomprimir el archivo {gz_file_path}: {e}")
        sys.exit()
else:
    print(f"El archivo {gz_file_path} no existe. No es posible continuar...")
    sys.exit()


## Bloque II
Carga del csv en pandas. Cambios de tipos e imputación

In [ ]:
df = pd.read_csv(output_file_path, low_memory=False)

In [25]:
df['price'] = df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df['price'] = df['price'].fillna(0)
df['available'] = df['available'].replace({'t': 1, 'f': 0})
df['minimum_nights'] = df['minimum_nights'].fillna(df['minimum_nights'].median()).astype(int)
df['maximum_nights'] = df['maximum_nights'].fillna(df['maximum_nights'].median()).astype(int)
df = df.drop(columns=['adjusted_price'])
df = df.rename(columns={'listing_id': 'property_id'})
# df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d')

C:\Users\dsantamaria\AppData\Local\Temp\ipykernel_8836\1973130110.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['available'] = df['available'].replace({'t': 1, 'f': 0})


In [26]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
property_id,9236806.0,6.438998e+17,5.357870e+17,21853.0,36760070.0,8.115293e+17,1.142566e+18,1.369179e+18
available,9236806.0,4.376801e-01,4.961011e-01,0.0,0.0,0.000000e+00,1.000000e+00,1.000000e+00
price,9236806.0,2.116178e+02,6.396718e+02,8.0,55.0,9.200000e+01,1.600000e+02,2.100000e+04
minimum_nights,9236806.0,1.016236e+01,3.081177e+01,1.0,1.0,2.000000e+00,6.000000e+00,1.125000e+03
maximum_nights,9236806.0,6.142410e+02,4.475243e+02,1.0,364.0,3.650000e+02,1.125000e+03,1.111100e+04


In [27]:
df.sample(10)

,property_id,date,available,price,minimum_nights,maximum_nights
4189440,711969127406978948,2025-11-04,1,188.0,5,365
4425800,766763084848115873,2025-04-24,0,37.0,7,1125
4117709,695618682919157245,2025-06-22,0,34.0,1,3
1871779,32395622,2025-08-01,0,57.0,2,1125
8005313,1252912175206179125,2025-04-28,1,95.0,2,365
4077089,685136472962266675,2025-08-10,0,64.0,30,180
3683215,592430604419656586,2026-01-11,1,337.0,99,1125
7504126,1193971208893539280,2025-05-02,1,65.0,4,365
5452600,939773237829340665,2025-04-02,1,136.0,30,365
2848291,44394920,2025-03-16,0,55.0,31,330


In [28]:
df.describe()

,property_id,available,price,minimum_nights,maximum_nights
count,9.236806e+06,9.236806e+06,9.236806e+06,9.236806e+06,9.236806e+06
mean,6.438998e+17,4.376801e-01,2.116178e+02,1.016236e+01,6.142410e+02
std,5.357870e+17,4.961011e-01,6.396718e+02,3.081177e+01,4.475243e+02
min,2.185300e+04,0.000000e+00,8.000000e+00,1.000000e+00,1.000000e+00
25%,3.676007e+07,0.000000e+00,5.500000e+01,1.000000e+00,3.640000e+02
50%,8.115293e+17,0.000000e+00,9.200000e+01,2.000000e+00,3.650000e+02
75%,1.142566e+18,1.000000e+00,1.600000e+02,6.000000e+00,1.125000e+03
max,1.369179e+18,1.000000e+00,2.100000e+04,1.125000e+03,1.111100e+04


## Bloque III
Importamos a la base de datos. 
**Requiere haber creado la tabla** <code>01_CREATE TABLE bnb_calendar.sql</code>

In [29]:
from config_bd import get_connection

In [30]:
df.to_csv(os.path.join(TEMP_DIR,'transform_calendar.csv'), index=False)

In [ ]:
from tqdm import tqdm
from time import sleep

CHUNKSIZE = 10000
for chunk in tqdm(
    pd.read_csv(os.path.join(TEMP_DIR, 'transform_calendar.csv'), chunksize=CHUNKSIZE, dtype=str),
    unit_scale=CHUNKSIZE, unit=" filas"
):
    sleep(5)




1700000 filas [01:30, 18697.33 filas/s]


KeyboardInterrupt: 

Elimino el fichero para no dejar duplicado el csv

In [ ]:
os.remove(os.path.join(TEMP_DIR,'transform_calendar.csv'))